In [1]:
import ipynb.fs.defs.functions_v2 as fct
import pandas as pd
import numpy as np
import pickle
import random
from scipy.stats import pearsonr, bernoulli

# Sim - refit

In [2]:
all_users_folder = 'data/all_users/'

# Load IDs
with open('uniqueIDs.pkl', 'rb') as f:
    uniqueIDs = pickle.load(f)

In [3]:
# General
model = 'mod0'

# Number of simulations
Nsim = 1

In [13]:
# Model settings
mod_info = pd.read_pickle(all_users_folder + model + '/mod_parameters.pkl')
print(mod_info)

# Extract functions
value_fct = 'rescorla_wagner_noV0_1trial'
dec_fct = 'my_softmax_1trial'

# Parameter bounds
param_lower_bounds = [0, 0]
param_upper_bounds = [1, 2]

{'name': 'model0', 'value_fct': 'rescorla_wagner_noV0', 'dec_fct': 'my_softmax', 'param_names': ['alpha', 'beta']}


In [5]:
def rescorla_wagner_noV0_1trial(vt_m_1, isHit, fb, alpha):    

    pe = np.nan

    # if hit, recieves fb
    if isHit == 1:
        # Compute prediction error
        pe = fb - vt_m_1
        # Compute new vt and fill in 
        vt = vt_m_1 + alpha * pe

    # if no hit, no fb
    elif isHit == 0:
        # vt does not change 
        vt = vt_m_1

    return vt, pe

In [6]:
def my_softmax_1trial(beta, vt):

    x = beta * vt
    
    p_hit =  np.exp(x)/(np.exp(x)+1)

    isHit = bernoulli.rvs(p_hit, size=1)

    return int(isHit)

In [7]:
def simulate_behav(fbs_all_cues, trialNo_all_cues, param_values, param_names):
    
    v0 = 0
        
    # Value function 
    # Free parameters
    alpha = param_values[param_names.index('alpha')]
    
    Ntrials = sum(len(lst) for lst in fbs_all_cues.values())

    # Initialise empty dictionary
    vt_all_cues = dict.fromkeys(fbs_all_cues.keys())
    pe_all_cues = dict.fromkeys(fbs_all_cues.keys())
    
    # Decision function
    # Free parameters
    beta = param_values[param_names.index('beta')]

    # Initialise empty dictionary
    isHit_all_cues = dict.fromkeys(vt_all_cues.keys())
    
    # Iterate over cues
    for cue, fbs in zip(fbs_all_cues.keys(), fbs_all_cues.values()):

        # Initialise vector of values
        vts = np.empty(len(fbs))
        vts.fill(np.nan)

        # Initialise vector of hits
        hits = np.empty(len(fbs))
        hits.fill(np.nan)

        # Initialise vector of PEs
        pes = np.empty(len(fbs))
        pes.fill(np.nan)

        # Fill in prior
        # If want specific prior per cue, check value of cue
        vts[0] = v0
        hits[0] = 0

        # Iterate to fill in vector
        for t in range(1,len(vts)):
            vt, pe = rescorla_wagner_noV0_1trial(vts[t-1], hits[t-1], fbs[t-1], alpha)
            isHit = my_softmax_1trial(beta, vt)
            vts[t] = vt
            pes[t] = pe
            hits[t] = isHit

        vt_all_cues[cue] = vts
        pe_all_cues[cue] = pes
        isHit_all_cues[cue] = hits
        
    return vt_all_cues, pe_all_cues, isHit_all_cues

In [10]:
# Get parameter values for simulation

# Load fitted parameters
data_mod = pd.read_pickle(all_users_folder + model + '/mod_param_fits.pkl')
parameter_means = data_mod[mod_info['param_names']].mean()
parameter_stds = data_mod[mod_info['param_names']].std()

# Initialise matrices of nans
all_sim_param_values = np.empty((Nsim, len(mod_info['param_names'])))
all_fit_param_values = np.empty((Nsim, len(mod_info['param_names'])))
all_sim_param_values[:] = np.NaN
all_fit_param_values[:] = np.NaN

# Sim and refit
for sim_id in range(0, Nsim):
    
    if sim_id%25==0:
        print(sim_id)
    
    # Sample from mean of fitted values
    params = np.array([])

    for param_mean, param_std, low_b, upp_b in zip(parameter_means, parameter_stds, param_lower_bounds, param_upper_bounds):

        # Sample parameter from mean (resample if not within the bounds)
        param = np.nan
        #while not(param >= low_b and param <= upp_b):
            #param = np.random.normal(size = 1, loc = param_mean, scale = param_std)
        param = np.random.uniform(low=low_b, high=upp_b, size=None)

        # Parameter value
        params = np.append(params, param)

    # Store
    mod_info['param_values'] = params

    # Create a model object
    mod = fct.Model(mod_name = mod_info['name'],
                 value_fct = value_fct, 
                 dec_fct = dec_fct, 
                 param_names = mod_info['param_names'])

    # Extract random data set
    random.shuffle(uniqueIDs)
    ID = uniqueIDs[0]
    user_folder = 'data/user_' + ID + '/'
    df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
    _, fbs_all_cues, trialNo_all_cues = fct.extract_hits_fbs(df2_cf)

    # Simulate behaviour
    #mod.simulate_behaviour(fbs_all_cues, trialNo_all_cues, mod_info['param_values'], mod_info['param_names'])
    vt_all_cues, pe_all_cues, isHit_all_cues = simulate_behav(fbs_all_cues, trialNo_all_cues, mod_info['param_values'], mod_info['param_names'])
    
    # Input data to model
    mod.gen_param_values = mod_info['param_values']
    mod.set_data('', fbs_all_cues, isHit_all_cues, trialNo_all_cues)    

    # Fit model
    mod.fit(param_lower_bounds, param_upper_bounds, n_iterations=8)

    # Store
    all_sim_param_values[sim_id,:] = mod.gen_param_values
    all_fit_param_values[sim_id,:] = mod.param_values

# Reformat
df_sim = pd.DataFrame(all_sim_param_values, columns = mod_info['param_names'])
df_sim.insert(0, 'Type', 'Sim')
df_sim = df_sim.rename_axis('simID').reset_index()

df_fit = pd.DataFrame(all_fit_param_values, columns = mod_info['param_names'])
df_fit.insert(0, 'Type', 'Fit')
df_fit = df_fit.rename_axis('simID').reset_index()

df = pd.concat([df_sim, df_fit])
df.sort_values(by='simID', ascending=True, inplace = True)
df.reset_index(inplace = True, drop = True)

# Save
df.to_pickle(all_users_folder + '/sim_refit/' + mod_info['name'] + '.pkl')

0


In [11]:
df

,simID,Type,alpha,beta
0,0,Sim,0.094301,1.085852
1,0,Fit,0.381966,0.763932
